In [1]:
import numpy as np
import re
import pandas as pd
import pickle
from collections import Counter
from openpyxl import Workbook
from openpyxl import load_workbook

In [2]:
B_df = pd.read_pickle("../data/5dimensions.pickle")
B_df

,say_id,reply_id,group_id,name,body,argument_a,argument_b,epistemic_a,epistemic_b,social_a,social_b,coordination_a,coordination_b
0,1,-1,1,まこぴす,よろしくお願いします！,non-argumentative moves,non-argumentative moves,off task,off task,blank,blank,blank,blank
1,31,-1,1,哲,よろしくお願いします,non-argumentative moves,non-argumentative moves,off task,off task,blank,blank,blank,blank
2,70,-1,1,仙波,名前なのが恥ずかしいです…\n\nよろしくお願いします！,non-argumentative moves,non-argumentative moves,off task,off task,blank,blank,blank,blank
3,119,-1,1,まこぴす,早速課題やっちゃいましょう！,non-argumentative moves,non-argumentative moves,off task,off task,blank,blank,proceedings,proceedings
4,163,-1,1,仙波,やっちゃいましょう\n\nmoodleはゴミです！,simple claim,simple claim,on task,on task,externalization,blank,blank,blank
5,194,-1,1,まこぴす,使いにくいです(笑),simple claim,simple claim,on task,on task,externalization,blank,blank,blank
6,302,-1,1,哲,同意です,simple claim,simple claim,on task,on task,quick consensus building,quick consensus building,blank,blank
7,309,-1,1,仙波,以前インタラクティブアート受講していたのですが、その時に課題が不具合で出せなくなっていた時期...,grounded claim,grounded claim,on task,on task,externalization,externalization,blank,blank
8,385,-1,1,まこぴす,学習環境として必要最低限の機能は備えていると思うが、操作性の面ではPCの最低限の知識があるこ...,grounded and qualified claim,grounded and qualified claim,on task,on task,externalization,externalization,blank,blank
9,426,-1,1,まこぴす,自分の意見です！(笑),non-argumentative moves,simple claim,off task,on task,blank,externalization,blank,blank


In [3]:
print("全データ：", B_df.shape,"\n")
drop_B_df = B_df.drop_duplicates(subset=['body', 'name']).reset_index(drop=True)
print("重複投稿を排除したデータ：", drop_B_df.shape,"\n")


valid = 0


argu_B_df = drop_B_df[drop_B_df['argument_a'] != 'blank'].reset_index(drop=True)
argu_B_df = argu_B_df[argu_B_df['argument_b'] != 'blank'].reset_index(drop=True)
print("argument有効データ(重複削除)：",argu_B_df.shape)
valid = argu_B_df.shape[0]
argu_B_df = argu_B_df[argu_B_df['argument_a'] == argu_B_df['argument_b']].reset_index(drop=True)
print("argument一致データ(重複削除)：",argu_B_df.shape)
print("argument一致率：",valid,"/",argu_B_df.shape[0],"=",argu_B_df.shape[0]/valid)
print(Counter(argu_B_df['argument_a']),"\n")



epis_B_df = drop_B_df[drop_B_df['epistemic_a'] != 'blank'].reset_index(drop=True)
epis_B_df = epis_B_df[epis_B_df['epistemic_b'] != 'blank'].reset_index(drop=True)
print("epistemic有効データ(重複削除)：",epis_B_df.shape)
valid = epis_B_df.shape[0]
epis_B_df = epis_B_df[epis_B_df['epistemic_a'] == epis_B_df['epistemic_b']].reset_index(drop=True)
print("epistemic一致データ(重複削除)：",epis_B_df.shape)
print("epistemic一致率：",valid,"/",epis_B_df.shape[0],"=",epis_B_df.shape[0]/valid)
print(Counter(epis_B_df['epistemic_a']),"\n")

soci_B_df = epis_B_df[epis_B_df['epistemic_a'] == 'on task'].reset_index(drop=True)
soci_B_df = soci_B_df[soci_B_df['epistemic_b'] == 'on task'].reset_index(drop=True)
print("social有効データ(重複削除)：",soci_B_df.shape)
valid = soci_B_df.shape[0]
soci_B_df = soci_B_df[soci_B_df['social_a'] == soci_B_df['social_b']].reset_index(drop=True)
print("social一致データ(重複削除)：",soci_B_df.shape)
print("social一致率：",valid,"/",soci_B_df.shape[0],"=",soci_B_df.shape[0]/valid)
print(Counter(soci_B_df['social_a']),"\n")


coor_B_df = epis_B_df[epis_B_df['epistemic_a'] == 'off task'].reset_index(drop=True)
coor_B_df = coor_B_df[coor_B_df['epistemic_b'] == 'off task'].reset_index(drop=True)
print("coordination有効データ(重複削除)：",coor_B_df.shape)
valid = coor_B_df.shape[0]
coor_B_df = coor_B_df[coor_B_df['coordination_a'] == coor_B_df['coordination_b']].reset_index(drop=True)
print("coordination一致データ(重複削除)：",coor_B_df.shape)
print("coordination一致率：",valid,"/",coor_B_df.shape[0],"=",coor_B_df.shape[0]/valid)
print(Counter(coor_B_df['coordination_a']),"\n")


全データ： (9959, 13) 

重複投稿を排除したデータ： (9320, 13) 

argument有効データ(重複削除)： (9057, 13)
argument一致データ(重複削除)： (7739, 13)
argument一致率： 9057 / 7739 = 0.8544771999558353
Counter({'non-argumentative moves': 4874, 'simple claim': 2621, 'grounded claim': 228, 'qualified claim': 13, 'grounded and qualified claim': 3}) 

epistemic有効データ(重複削除)： (9315, 13)
epistemic一致データ(重複削除)： (8451, 13)
epistemic一致率： 9315 / 8451 = 0.9072463768115943
Counter({'off task': 4522, 'on task': 3916, 'no sense': 13}) 

social有効データ(重複削除)： (3916, 13)
social一致データ(重複削除)： (2604, 13)
social一致率： 3916 / 2604 = 0.6649642492339122
Counter({'externalization': 1285, 'quick consensus building': 857, 'elicitation': 275, 'integration-oriented consensus building': 102, 'conflict-oriented consensus building': 63, 'summary': 20, 'blank': 2}) 

coordination有効データ(重複削除)： (4522, 13)
coordination一致データ(重複削除)： (3235, 13)
coordination一致率： 4522 / 3235 = 0.715391419725785
Counter({'blank': 2229, 'technical coordination': 711, 'proceedings': 226, 'time manag

In [4]:
All_df = pd.read_pickle("../data/all.pickle")
All_df

,say_id,reply_id,group_id,name,body,16types_a,16types_b,argument_a,argument_b,epistemic_a,epistemic_b,social_a,social_b,coordination_a,coordination_b
0,1,-1,1234568,まこぴす,よろしくお願いします！,5,5,1,1,1,1,0,0,0,0
1,31,-1,1234568,哲,よろしくお願いします,5,5,1,1,1,1,0,0,0,0
2,70,-1,1234568,仙波,名前なのが恥ずかしいです…\nよろしくお願いします！,5,5,1,1,1,1,0,0,0,0
3,119,-1,1234568,まこぴす,早速課題やっちゃいましょう！,14,14,1,1,1,1,0,0,2,2
4,163,-1,1234568,仙波,やっちゃいましょう\nmoodleはゴミです！,1,2,2,2,2,2,1,0,0,0
5,194,-1,1234568,まこぴす,使いにくいです(笑),2,2,2,2,2,2,1,0,0,0
6,302,-1,1234568,哲,同意です,1,1,2,2,2,2,2,2,0,0
7,309,-1,1234568,仙波,以前インタラクティブアート受講していたのですが、その時に課題が不具合で出せなくなっていた時期...,2,2,3,3,2,2,1,1,0,0
8,385,-1,1234568,まこぴす,学習環境として必要最低限の機能は備えていると思うが、操作性の面ではPCの最低限の知識があるこ...,2,2,5,5,2,2,1,1,0,0
9,426,-1,1234568,まこぴす,自分の意見です！(笑),2,2,1,2,1,2,0,1,0,0


In [5]:
print("全データ：", All_df.shape,"\n")
drop_All_df = All_df.drop_duplicates(subset=['body', 'name']).reset_index(drop=True)
print("重複投稿を排除したデータ：", drop_All_df.shape,"\n")


valid = 0

t_All_df = drop_All_df
print("16types有効データ(重複削除)：",t_All_df.shape)
valid = t_All_df.shape[0]
t_All_df = t_All_df[t_All_df['16types_a'] == t_All_df['16types_b']].reset_index(drop=True)
print("16types一致データ(重複削除)：",t_All_df.shape)
print("16types一致率：",valid,"/",t_All_df.shape[0],"=",t_All_df.shape[0]/valid)
print(Counter(t_All_df['16types_a']),"\n")


x_All_df = drop_All_df
x_All_df = x_All_df[x_All_df['16types_a'] == x_All_df['16types_b']].reset_index(drop=True)
x_All_df = x_All_df[x_All_df['argument_a'] == x_All_df['argument_b']].reset_index(drop=True)
x_All_df = x_All_df[x_All_df['epistemic_a'] == x_All_df['epistemic_b']].reset_index(drop=True)
x_All_df = x_All_df[x_All_df['social_a'] == x_All_df['social_b']].reset_index(drop=True)
x_All_df = x_All_df[x_All_df['coordination_a'] == x_All_df['coordination_b']].reset_index(drop=True)
print("全ラベル一致データ(重複削除)：",x_All_df.shape,"\n")



argu_All_df = drop_All_df[drop_All_df['argument_a'] != 0].reset_index(drop=True)
argu_All_df = argu_All_df[argu_All_df['argument_b'] != 0].reset_index(drop=True)
print("argument有効データ(重複削除)：",argu_All_df.shape)
valid = argu_All_df.shape[0]
argu_All_df = argu_All_df[argu_All_df['argument_a'] == argu_All_df['argument_b']].reset_index(drop=True)
print("argument一致データ(重複削除)：",argu_All_df.shape)
print("argument一致率：",valid,"/",argu_All_df.shape[0],"=",argu_All_df.shape[0]/valid)
print(Counter(argu_All_df['argument_a']),"\n")

epis_All_df = drop_All_df[drop_All_df['epistemic_a'] != 0].reset_index(drop=True)
epis_All_df = epis_All_df[epis_All_df['epistemic_b'] != 0].reset_index(drop=True)
print("epistemic有効データ(重複削除)：",epis_All_df.shape)
valid = epis_All_df.shape[0]
epis_All_df = epis_All_df[epis_All_df['epistemic_a'] == epis_All_df['epistemic_b']].reset_index(drop=True)
print("epistemic一致データ(重複削除)：",epis_All_df.shape)
print("epistemic一致率：",valid,"/",epis_All_df.shape[0],"=",epis_All_df.shape[0]/valid)
print(Counter(epis_All_df['epistemic_a']),"\n")

soci_All_df = epis_All_df[epis_All_df['epistemic_a'] == 2].reset_index(drop=True)
soci_All_df = soci_All_df[soci_All_df['epistemic_b'] == 2].reset_index(drop=True)
print("social有効データ(重複削除)：",soci_All_df.shape)
valid = soci_All_df.shape[0]
soci_All_df = soci_All_df[soci_All_df['social_a'] == soci_All_df['social_b']].reset_index(drop=True)
print("social一致データ(重複削除)：",soci_All_df.shape)
print("social一致率：",valid,"/",soci_All_df.shape[0],"=",soci_All_df.shape[0]/valid)
print(Counter(soci_All_df['social_a']),"\n")


coor_All_df = epis_All_df[epis_All_df['epistemic_a'] == 1].reset_index(drop=True)
coor_All_df = coor_All_df[coor_All_df['epistemic_b'] == 1].reset_index(drop=True)
print("coordination有効データ(重複削除)：",coor_All_df.shape)
valid = coor_All_df.shape[0]
coor_All_df = coor_All_df[coor_All_df['coordination_a'] == coor_All_df['coordination_b']].reset_index(drop=True)
print("coordination一致データ(重複削除)：",coor_All_df.shape)
print("coordination一致率：",valid,"/",coor_All_df.shape[0],"=",coor_All_df.shape[0]/valid)
print(Counter(coor_All_df['coordination_a']),"\n")

全データ： (12012, 15) 

重複投稿を排除したデータ： (11357, 15) 

16types有効データ(重複削除)： (11357, 15)
16types一致データ(重複削除)： (7926, 15)
16types一致率： 11357 / 7926 = 0.6978955710134719
Counter({1: 1867, 2: 1325, 5: 1046, 3: 900, 4: 885, 6: 779, 8: 236, 7: 227, 14: 179, 9: 141, 11: 108, 15: 102, 12: 63, 13: 30, 0: 21, 10: 17}) 

全ラベル一致データ(重複削除)： (5417, 15) 

argument有効データ(重複削除)： (9057, 15)
argument一致データ(重複削除)： (7739, 15)
argument一致率： 9057 / 7739 = 0.8544771999558353
Counter({1: 4874, 2: 2621, 3: 228, 4: 13, 5: 3}) 

epistemic有効データ(重複削除)： (9315, 15)
epistemic一致データ(重複削除)： (8451, 15)
epistemic一致率： 9315 / 8451 = 0.9072463768115943
Counter({1: 4522, 2: 3916, 3: 13}) 

social有効データ(重複削除)： (3916, 15)
social一致データ(重複削除)： (2604, 15)
social一致率： 3916 / 2604 = 0.6649642492339122
Counter({1: 1285, 2: 857, 3: 275, 4: 102, 5: 63, 6: 20, 0: 2}) 

coordination有効データ(重複削除)： (4522, 15)
coordination一致データ(重複削除)： (3235, 15)
coordination一致率： 4522 / 3235 = 0.715391419725785
Counter({0: 2229, 1: 711, 2: 226, 4: 36, 5: 21, 3: 12}) 

